In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
wine=pd.read_csv('https://bit.ly/wine_csv_data')

data=wine[['alcohol', 'sugar', 'pH']].to_numpy()
target=wine['class'].to_numpy()

X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)

In [14]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf= RandomForestClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(rf,X_train,y_train,return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']),np.mean(scores['test_score']))


0.9973541965122431 0.8905151032797809


In [15]:
rf.fit(X_train,y_train)
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

In [16]:
#oob_score_ : 부트스트랩 샘플에 포함되지 않고 남는 훈련데이터의 스코어
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(X_train,y_train)
print(rf.oob_score_)


0.8934000384837406


In [17]:
#Extra _Tree : bootstrap을 사용하지 않음 (전체훈련세트 사용)(중복허용x)(무작위 분할 = 분류기를 랜덤하게), 랜덤포레스트와 비슷하다
#spliter== random , but randomforestclassifier  spliter==best

from sklearn.ensemble import ExtraTreesClassifier

et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,X_train,y_train,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9974503966084433
0.8887848893166506


In [18]:
et.fit(X_train,y_train)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [21]:
#그레디언트 부스팅 : 깊이가 얉은 결정트리를 사용 (기본적으로 깊이 3, 트리 100개) , 때문에 과대적합에 강하다
#경사 하강법을 사용하여 트리를 앙상블에 추가.
#return_train_score : 검증 뿐만 아닌 훈련세트의 스코어

from sklearn.ensemble import GradientBoostingClassifier

gd=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gd,X_train,y_train,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))
print(scores)


0.8881086892152563
0.8720430147331015
{'fit_time': array([0.24235654, 0.24534941, 0.34674048, 0.3467412 , 0.34973478]), 'score_time': array([0.00199485, 0.00302458, 0.00299311, 0.00299287, 0.00398946]), 'test_score': array([0.86634615, 0.87019231, 0.89412897, 0.86044273, 0.86910491]), 'train_score': array([0.89006495, 0.88958383, 0.88239538, 0.89249639, 0.88600289])}


In [25]:
#트리 100개에서 500개로 늘린 score
from sklearn.ensemble import GradientBoostingClassifier

gd=GradientBoostingClassifier(n_estimators=500,random_state=42,learning_rate=0.3)
scores=cross_validate(gd,X_train,y_train,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9574268869433652
0.8778161323758052


In [28]:
#히스토그램 기반 그레이디언트 부스팅 : 입력 특성을 256개의 구간으로 나눈다. 256개의 구간중에서 하나를 떼어놓고 누락된 값을 위해서 사용한다

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb,X_train,y_train,return_train_score=True)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

#과대적합을 잘 억제하면서 그레이디언트 부스팅 보다 조금 더 높은 성능을 제공.

0.9321723946453317
0.8801241948619236


In [30]:
from sklearn.inspection import permutation_importance  #특성중요도

hgb.fit(X_train,y_train)
result(permutation_importance(hgb,X_train,y_train,n_repeats=10, random_state=42,n_jobs=-1))        #n_repeats = 섞는 횟수

<bound method BaseEstimator.get_params of HistGradientBoostingClassifier(random_state=42)>